In [34]:
# http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210302A1.ipynb
prfx_prp = 'prep_20210302B1'

In [35]:
import pandas as pd
import numpy as np
from collections import Counter


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier




In [36]:
MIN_CNT = 50

In [47]:
%%time
# df = pd.read_csv(f'../data/data__{prfx_prp}.csv')

df = pd.read_csv(f'../data/data_sample__{prfx_prp}.csv')
df['category'] = df.category.apply(eval)
df.fillna("", inplace=True)
print("df.shape:", df.shape)
df['txt'] = df.title + " " + df.brand + " " + df.description + " " + df.feature

df.shape: (10000, 7)
CPU times: user 238 ms, sys: 3.09 ms, total: 242 ms
Wall time: 240 ms


In [48]:
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter([j for i in df.category for j in i])
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

len(i2dmn), len(i2cat)

(23, 111)

## split train val

In [49]:
np.random.seed(101)
msk_val = np.random.rand(len(df))>0.85
dftrn = df[~msk_val]
dfval = df[msk_val]
dftrn.shape, dfval.shape

((8466, 8), (1534, 8))

## rf model

In [54]:
%%time
vec = TfidfVectorizer(ngram_range=(1,2),
                      min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
Xtrn = vec.fit_transform(dftrn.txt)
Xval = vec.transform(dfval.txt)

Xtrn.shape, Xval.shape

CPU times: user 2.94 s, sys: 29.4 ms, total: 2.97 s
Wall time: 2.97 s


((8466, 57858), (1534, 57858))

In [57]:
df['cat'] = df.category.apply(lambda x: [cat2i[o]])

0       [Clothing, Shoes & Jewelry, Women, Contemporar...
1       [Tools & Home Improvement, Measuring & Layout ...
2       [Clothing, Shoes & Jewelry, Novelty & More, Cl...
3       [Home & Kitchen, Kitchen & Dining, Dining & En...
4       [Automotive, Motorcycle & Powersports, Protect...
                              ...                        
9995    [Clothing, Shoes & Jewelry, Novelty & More, Cl...
9996         [Books, Literature & Fiction, Genre Fiction]
9997    [Tools & Home Improvement, Power & Hand Tools,...
9998    [Electronics, Computers & Accessories, Network...
9999    [Clothing, Shoes & Jewelry, Women, Shoes, Boot...
Name: category, Length: 10000, dtype: object

In [77]:
ys = np.zeros((len(df), len(i2cat)))

for i,cats in enumerate(df.category):
    idx_pos = [cat2i[cat] for cat in cats if cat in cat2i]
    ys[i,idx_pos] = 1

ys_trn = ys[~msk_val]
ys_val = ys[msk_val]

In [79]:
clf = RandomForestClassifier(n_estimators=5, 
                             min_samples_leaf=2,
                             max_features=0.5,
                             n_jobs=-1)

In [80]:
%%time
clf.fit(Xtrn, ys_trn)

RandomForestClassifier(max_features=0.5, min_samples_leaf=2, n_estimators=5,
                       n_jobs=-1)

In [82]:
clf.n_outputs_

111

In [97]:
prds_trn = clf.predict(Xtrn)
prds_prob_trn = clf.predict_proba(Xtrn)

np.mean(prds_trn==ys_trn), accuracy_score(ys_trn, prds_trn)

(0.9940620989522477, 0.591896999763761)

In [98]:
prds_val = clf.predict(Xval)
prds_prob_val = clf.predict_proba(Xval)

np.mean(prds_val==ys_val), accuracy_score(ys_val, prds_val)

(0.9859755452975792, 0.2255541069100391)

In [99]:
for j in range(ys_val.shape[1]):
    print(accuracy_score(ys_val[:,j], prds_val[:,j]))

0.9934810951760105
0.9973924380704041
0.9934810951760105
0.9511082138200783
0.9915254237288136
0.9954367666232073
0.9928292046936115
0.9908735332464146
0.9850065189048239
0.9934810951760105
0.9915254237288136
0.9621903520208605
0.9973924380704041
0.9960886571056062
0.9980443285528031
0.9856584093872229
0.9882659713168188
0.9941329856584094
0.9934810951760105
0.9322033898305084
0.9941329856584094
0.9947848761408083
0.9967405475880052
0.9980443285528031
0.9732724902216427
0.9856584093872229
0.9850065189048239
0.9947848761408083
0.983702737940026
0.9797913950456323
0.9941329856584094
0.9550195567144719
0.9680573663624511
0.9941329856584094
0.9817470664928292
0.9928292046936115
0.9954367666232073
0.9941329856584094
0.9967405475880052
0.9524119947848761
0.9941329856584094
0.9869621903520208
0.9882659713168188
0.9947848761408083
0.9908735332464146
0.9928292046936115
0.9915254237288136
0.9947848761408083
0.9921773142112125
0.9960886571056062
0.9954367666232073
0.9954367666232073
0.98565840938